In [2]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct
import os

def pad_image(image, block_size):
    width, height = image.size
    
    # Calculate the number of blocks in each dimension
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size
    
    # Calculate the padded dimensions
    padded_width = (num_blocks_x + 1) * block_size
    padded_height = (num_blocks_y + 1) * block_size
    
    # Create a new blank image with the padded dimensions
    padded_image = Image.new('L', (padded_width, padded_height))
    
    # Paste the original image onto the padded image
    padded_image.paste(image, (0, 0))
    
    return padded_image


def calculate_compression_ratio(original_size, compressed_size):
    compression_ratio = original_size / compressed_size
    return compression_ratio

In [3]:


def compress_image(image_path, block_size, target_rmse, truncation_percentage):
    # Load the image and convert it to grayscale
    image = Image.open(image_path).convert('L')

    # Pad the image to match the block size
    padded_image = pad_image(image, block_size)

    # Get the padded image dimensions
    width, height = padded_image.size

    # Calculate the number of blocks in each dimension
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size

    # Initialize an array to store the compressed image
    compressed_image = np.zeros((height, width))

    # Iterate over each block in the image
    for y in range(num_blocks_y):
        for x in range(num_blocks_x):
            # Extract the current block from the image
            block = padded_image.crop((x * block_size, y * block_size,
                                (x + 1) * block_size, (y + 1) * block_size))

            # Convert the block to a NumPy array and apply DCT
            block_arr = np.array(block, dtype=np.float32)
            dct_coefficients = dct(dct(block_arr.T, norm='ortho').T, norm='ortho')

            # Calculate the number of coefficients to keep based on truncation percentage
            num_coeffs = int((100 - truncation_percentage) / 100 * block_size**2)

            # Sort the coefficients in descending order of magnitude
            sorted_coeffs = np.abs(dct_coefficients).ravel()
            sorted_coeffs.sort()

            # Determine the threshold value
            threshold = sorted_coeffs[-num_coeffs]

            # Discard high-frequency coefficients by applying truncation
            dct_coefficients[np.abs(dct_coefficients) < threshold] = 0

            # Reconstruct the block by applying inverse DCT
            reconstructed_block = idct(idct(dct_coefficients.T, norm='ortho').T, norm='ortho')

            # Store the reconstructed block in the compressed image
            compressed_image[y * block_size:(y + 1) * block_size, x * block_size:(x + 1) * block_size] = reconstructed_block

    # Calculate the RMSE between the original and compressed images
    original_image_arr = np.array(padded_image, dtype=np.float32)
    compressed_image_arr = compressed_image[:height, :width]
    rmse = np.sqrt(np.mean((original_image_arr - compressed_image_arr) ** 2))

    # Check if the RMSE is below the target threshold
    if rmse > target_rmse:
        print(f"Compression failed. RMSE: {rmse:.2f}")
    else:
        print(f"Compression successful. RMSE: {rmse:.2f}")

        # Save the compressed image
        compressed_image = Image.fromarray(compressed_image_arr.astype(np.uint8))
        compressed_image.save("compressed_image.jpg")

        
        compressed_file_size = os.stat("compressed_image.jpg").st_size
        original_file_size = os.stat(image_path).st_size
        ratio = calculate_compression_ratio(original_file_size, compressed_file_size)
        print(f"Compression Ratio: {ratio:.2f}")

In [5]:
# test usage
compress_image("q3.jpg", block_size=64, target_rmse=1.0, truncation_percentage=10)


Compression successful. RMSE: 0.38
Compression Ratio: 4.44
